In [2]:
from stackapi import StackAPI
import pandas as pd
import time
SITE = StackAPI('stackoverflow')

In [3]:
start_time = int(time.mktime(time.strptime("2016-06-01T00:00:00", "%Y-%m-%dT%H:%M:%S")))
end_time = int(time.mktime(time.strptime("2017-06-01T00:00:00", "%Y-%m-%dT%H:%M:%S")))
questions = SITE.fetch('questions', fromdate=start_time, todate=end_time,
                       min=20, sort='votes')

In [4]:
qdf = pd.DataFrame.from_dict(questions)
i = 2
while questions['has_more'] == True and i < 10:
    questions = SITE.fetch('questions', fromdate=start_time, todate=end_time,
                       min=20, sort='votes', page=i)
    tdf = pd.DataFrame.from_dict(questions)
    qdf = pd.concat([qdf, tdf])
    i += 1 
qdf = qdf.reset_index()
qdf = qdf.to_csv("questions.csv")

In [5]:
import ast
answerlist = []
qdf = pd.read_csv("questions.csv")
for index, row in qdf.iterrows():
    rd = ast.literal_eval(row['items'])
    if 'accepted_answer_id' in rd and 'user_id' in rd['owner']:
        answerlist.append(rd['accepted_answer_id'])

In [8]:
adf = {}
numiters = int(len(answerlist) / 99)
if numiters == 0:
    answers = SITE.fetch('/answers/{ids}', ids=answerlist)
    adf = pd.DataFrame.from_dict(answers)
else:
    answers = SITE.fetch('/answers/{ids}', ids=answerlist[:99])
    adf = pd.DataFrame.from_dict(answers)
    j = 99
    k = j + 99
    for i in range(0, numiters):
        answers = SITE.fetch('/answers/{ids}', ids=answerlist[j:k])
        temp = pd.DataFrame.from_dict(answers)
        adf = pd.concat([adf, temp])
        j = j + 99
        k = k + 99
adf = adf.reset_index()
adf = adf.to_csv("answers.csv")

In [9]:
#answers = SITE.fetch('/answers/{ids}', ids=answerlist)
#adf = pd.DataFrame.from_dict(answers)
#i = 2
#while answers['has_more'] == True and i <= 50:
#    answers = SITE.fetch('/answers/{ids}', ids=answerlist, page=i)
#    tdf = pd.DataFrame.from_dict(answers)
#    adf = pd.concat([adf, tdf])
#    i += 1 
#adf = adf.reset_index()
#adf = adf.to_csv("answers.csv")

In [1]:
#Only run above when needed.
import ast
import pandas as pd
userasklist = []
useranswerlist = []
questionlist = []
adf = pd.read_csv("answers.csv")
qdf = pd.read_csv("questions.csv")
dq = {}
dtag = {}
for index, row in qdf.iterrows():
    q = ast.literal_eval(row['items'])
    if 'accepted_answer_id' in q and 'user_id' in q['owner']:
        dq[q['question_id']] = q['owner']['user_id']
        if 'tags' in q:
            dtag[q['question_id']] = q['tags']
        else:
            dtag[q['question_id']] = []

In [25]:
from IPython.display import display
da = {}
for index, row in adf.iterrows():
    q = ast.literal_eval(row['items'])
    if 'user_id' in q['owner']:
        da[q['question_id']] = q['owner']['user_id']
    else:
        #find a way to delete this
        da[q['question_id']] = "Null"

d = {"asked": pd.Series(dq.values(), index=dq.keys()), 
     "answered": pd.Series(da.values(), index=dq.keys()),
     "tags":pd.Series(dtag.values(), index=dq.keys())}
data = pd.DataFrame(d)
data = data.reset_index()
data = data.rename(columns = {'index':'question'})
print display(data[:-10:-1])

,question,answered,asked,tags
1034,41400317,2380830,6307513,"[c++, memory, allocation]"
1033,38234107,5764320,5764320,"[osx, android-manifest, android-studio-2.1]"
1032,39753721,4495824,2832872,"[c++, std, ofstream]"
1031,42977270,6008532,2570103,"[c#, file, file-exists]"
1030,39348213,6303600,6303600,"[intellij-idea, angular, typescript, google-ch..."
1029,38740648,41116,5571327,"[ios, uiviewcontroller, swift3]"
1028,38436847,335858,5962841,"[c#, lazy-evaluation]"
1027,37613550,6420110,919445,"[ios, xcode]"
1026,37576685,1048572,4033215,"[javascript, node.js, promise, async-await, ec..."


None


In [77]:
import networkx as nx
g = nx.from_pandas_dataframe(data, "asked", "answered", edge_attr="tags")
dataasked = data[["asked","tags"]]
dataasked.columns = ["id","tags"]
dataanswered = data[["answered","tags"]]
dataanswered.columns = ["id","tags"]
tagcheck = pd.concat([dataasked, dataanswered])
tagcheck.set_index("id")
#print tagcheck.ix[:,0].tolist()
for n, sd in g.nodes_iter(True):
    if n in tagcheck.ix[:,0].tolist():
        sd['tags'] = tagcheck.loc[tagcheck["id"] == n , "tags"].values[0]

In [78]:
#calcualte diameter.  We must pick a subset that is entirely connected and check that diameter.  
#We will use the biggest network
bn = sorted(nx.connected_components(g), key=len, reverse=True)[0]
g = g.subgraph(bn)
print nx.diameter(g)
print sorted(nx.degree_centrality(g).iteritems(), key=lambda(k, v): (-v, k))[0:9]

6
[(440558, 0.2692307692307693), (1774667, 0.2692307692307693), (4342498, 0.2692307692307693), (4987285, 0.2692307692307693), (560648, 0.07692307692307693), (4624156, 0.07692307692307693), (6149078, 0.07692307692307693), (43534, 0.038461538461538464), (147192, 0.038461538461538464)]


In [79]:
#change graph to just the largest network
so_nodes = g.nodes()
so_edges = g.edges()

In [80]:
from graphlab import SGraph, Vertex, Edge, canvas
canvas.set_target('ipynb')
so_graph = SGraph()
nodelist = []
for i in so_nodes:
    nodelist.append(Vertex(i))
so_graph = so_graph.add_vertices(nodelist)
for i, j in so_edges:
    so_graph = so_graph.add_edges(Edge(i,j))
so_graph.show(vlabel='id', arrows=True)

In [71]:
print nx.get_node_attributes(g , 'tags')

{7140484: [u'c++', u'function-pointers', u'function-object', u'inlining'], 560648: [u'c++', u'c++14', u'return-type-deduction'], 4671117: [u'firebase', u'firebase-console'], 43534: [u'c++', u'c++11', u'templates', u'exception-handling', u'typetraits'], 2545680: [u'javascript'], 4987285: [u'c++', u'c++11', u'destructor'], 4624156: [u'android-studio', u'kotlin'], 1832221: [u'android', u'android-layout', u'preview', u'android-studio-2.2', u'android-layout-editor'], 5405086: [u'c++', u'memory-alignment', u'bit-fields', u'memory-layout', u'object-layout'], 853791: [u'c++', u'c++11', u'c++14', u'decltype', u'type-deduction'], 1296806: [u'scala'], 675100: [u'c++', u'string', u'c++1z', u'string-view'], 5835947: [u'java', u'exception', u'exception-handling'], 6920495: [u'android', u'android-support-design', u'bottomnavigationview'], 7867195: [u'java'], 1575613: [u'ios', u'itunesconnect'], 463300: [u'python', u'short-circuiting'], 179910: [u'c', u'semicolon', u'freeform'], 1774667: [u'c++', u'c+